**https://github.com/krishnaik06/Fake-New-LSTM/blob/master/FakeNewsClassifierUsingBidirectionalLSTM.ipynb**

**https://github.com/ArrasL/LRP_for_LSTM**

**Mount Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Import Libraries**

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import metrics
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs
#!pip install sckit-datasets
#from sklearn.datasets.base import Bunch
from sklearn import svm
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
!pip install lime
from lime.lime_text import LimeTextExplainer
from lime.lime_text import LimeTextExplainer
from lime import lime_text
import os
import sys
import numpy as np
from numpy import mean,std
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, NuSVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from lightgbm import LGBMClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import metrics
import itertools
from itertools import chain, product
from itertools import zip_longest
# importing libraries
import pandas as pd
import matplotlib.pyplot as plt
import string
import re
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import sklearn.metrics
# downloading the nltk data for preprocessing
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

     |████████████████████████████████| 275 kB 5.4 MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283857 sha256=6e73593ee2aeab9a6f1d2945be33d38b2267fa7b9227484c9876e31f2b48457e
  Stored in directory: /root/.cache/pip/wheels/ca/cb/e5/ac701e12d365a08917bf4c6171c0961bc880a8181359c66aa7
Successfully built lime
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


**Load Datasets**

In [ ]:
Train_data = pd.read_csv('/content/drive/MyDrive/Natural Language Processing/Fake News Detection/Conference Paper/Data/COVID-19/Constraint_Train.csv')
Test_data = pd.read_csv('/content/drive/MyDrive/Natural Language Processing/Fake News Detection/Conference Paper/Data/COVID-19/Constraint_Test.csv')
Val_data = pd.read_csv('/content/drive/MyDrive/Natural Language Processing/Fake News Detection/Conference Paper/Data/COVID-19/Constraint_Val.csv')

In [ ]:
Train_data['label'] = Train_data['label'].apply(lambda x: 1 if x == 'real' else 0)
Val_data['label'] = Val_data['label'].apply(lambda x: 1 if x == 'real' else 0)

In [ ]:
Train_data.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,1
1,2,States reported 1121 deaths a small rise from ...,1
2,3,Politically Correct Woman (Almost) Uses Pandem...,0
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,1
4,5,Populous states can generate large case counts...,1


In [ ]:
Val_data.head()

,id,tweet,label
0,1,Chinese converting to Islam after realising th...,0
1,2,11 out of 13 people (from the Diamond Princess...,0
2,3,"COVID-19 Is Caused By A Bacterium, Not Virus A...",0
3,4,Mike Pence in RNC speech praises Donald Trump’...,0
4,5,6/10 Sky's @EdConwaySky explains the latest #C...,1


In [ ]:
Test_data.head()

,id,tweet
0,1,Our daily update is published. States reported...
1,2,Alfalfa is the only cure for COVID-19.
2,3,President Trump Asked What He Would Do If He W...
3,4,States reported 630 deaths. We are still seein...
4,5,This is the sixth time a global health emergen...


In [ ]:
Train_data.shape, Val_data.shape, Test_data.shape

((6420, 3), (2140, 3), (2140, 2))

**Text Preprocessing**

In [ ]:
# NLP pre-processing
# remove urls, handles, and the hashtag from hashtags 
# (taken from https://stackoverflow.com/questions/8376691/how-to-remove-hashtag-user-link-of-a-tweet-using-regular-expression)
def remove_urls(text):
  new_text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split())
  return new_text

# make all text lowercase
def text_lowercase(text): 
  return text.lower()

# remove numbers
def remove_numbers(text): 
  result = re.sub(r'\d+', '', text) 
  return result

# remove punctuation
def remove_punctuation(text): 
  translator = str.maketrans('', '', string.punctuation)
  return text.translate(translator)

# function for all pre-processing steps
def preprocessing(text):
  text = text_lowercase(text)
  text = remove_urls(text)
  text = remove_numbers(text)
  text = remove_punctuation(text)
  return text

# pre-processing the text body column
pp_text = []
for text_data in Train_data['tweet']:
  # check if string
  if isinstance(text_data, str):
    pp_text_data = preprocessing(text_data)
    pp_text.append(pp_text_data)
   # if not string
  else:
    pp_text.append(np.NaN)

# add pre-processed column to dataset
Train_data['text'] = pp_text

In [ ]:
df = Train_data
df.head()

,id,tweet,label,text
0,1,The CDC currently reports 99031 deaths. In gen...,1,the cdc currently reports deaths in general t...
1,2,States reported 1121 deaths a small rise from ...,1,states reported deaths a small rise from last...
2,3,Politically Correct Woman (Almost) Uses Pandem...,0,politically correct woman almost uses pandemic...
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,1,indiafightscorona we have covid testing labor...
4,5,Populous states can generate large case counts...,1,populous states can generate large case counts...


In [ ]:
df['text'][3]

'indiafightscorona we have  covid testing laboratories in india and as on th august   tests have been done dg staysafe indiawillwin'

In [ ]:
output = []

for i in df['text']:
    output.append(i)

len(output)

6420

In [ ]:
from nltk.tokenize import word_tokenize

tokenized_sents = [word_tokenize(i) for i in output]

#**Creating own Vocab**

In [ ]:
import gensim
from gensim import corpora
from pprint import pprint
from gensim import models
from gensim.models import LdaModel
from gensim.models import TfidfModel
from gensim.models import CoherenceModel

In [ ]:
Embedding_Dim = 100

#train word2vec model
model = gensim.models.Word2Vec(sentences=tokenized_sents, size=Embedding_Dim, workers=4, min_count=1)

#Vocabulary size
myvocab = list(model.wv.vocab)

In [ ]:
#The total words present in our Bag of Words or Vocab size
print('Vocabulary Size : %d' % len(myvocab))

Vocabulary Size : 11601


#**Text LRP**

In [ ]:
!git clone https://github.com/ArrasL/LRP_for_LSTM.git

Cloning into 'LRP_for_LSTM'...
remote: Enumerating objects: 116, done.
remote: Total 116 (delta 0), reused 0 (delta 0), pack-reused 116
Receiving objects: 100% (116/116), 12.68 MiB | 23.87 MiB/s, done.
Resolving deltas: 100% (52/52), done.


In [ ]:
from LRP_for_LSTM import *

In [ ]:
!python /content/LRP_for_LSTM/code/LSTM/LRP_linear_layer.py

In [ ]:
!python /content/LRP_for_LSTM/code/LSTM/LSTM_bidi.py

Traceback (most recent call last):
  File "/content/LRP_for_LSTM/code/LSTM/LSTM_bidi.py", line 13, in <module>
    from code.LSTM.LRP_linear_layer import *
ModuleNotFoundError: No module named 'code.LSTM'; 'code' is not a package


In [ ]:
!python /content/LRP_for_LSTM/code/LSTM/__init__.py

In [ ]:
!python /content/LRP_for_LSTM/code/util/__init__.py

In [ ]:
!python /content/LRP_for_LSTM/code/util/heatmap.py

In [ ]:
!python /content/LRP_for_LSTM/code/__init__.py

**LRP_linear_layer.py**

In [ ]:
'''
@author: Leila Arras
@maintainer: Leila Arras
@date: 21.06.2017
@version: 1.0+
@copyright: Copyright (c) 2017, Leila Arras, Gregoire Montavon, Klaus-Robert Mueller, Wojciech Samek
@license: see LICENSE file in repository root
'''

import numpy as np
from numpy import newaxis as na


def lrp_linear(hin, w, b, hout, Rout, bias_nb_units, eps, bias_factor=0.0, debug=False):
    """
    LRP for a linear layer with input dim D and output dim M.
    Args:
    - hin:            forward pass input, of shape (D,)
    - w:              connection weights, of shape (D, M)
    - b:              biases, of shape (M,)
    - hout:           forward pass output, of shape (M,) (unequal to np.dot(w.T,hin)+b if more than one incoming layer!)
    - Rout:           relevance at layer output, of shape (M,)
    - bias_nb_units:  total number of connected lower-layer units (onto which the bias/stabilizer contribution is redistributed for sanity check)
    - eps:            stabilizer (small positive number)
    - bias_factor:    set to 1.0 to check global relevance conservation, otherwise use 0.0 to ignore bias/stabilizer redistribution (recommended)
    Returns:
    - Rin:            relevance at layer input, of shape (D,)
    """
    sign_out = np.where(hout[na,:]>=0, 1., -1.) # shape (1, M)
    
    numer    = (w * hin[:,na]) + ( bias_factor * (b[na,:]*1. + eps*sign_out*1.) / bias_nb_units ) # shape (D, M)
    # Note: here we multiply the bias_factor with both the bias b and the stabilizer eps since in fact
    # using the term (b[na,:]*1. + eps*sign_out*1.) / bias_nb_units in the numerator is only useful for sanity check
    # (in the initial paper version we were using (bias_factor*b[na,:]*1. + eps*sign_out*1.) / bias_nb_units instead)
    
    denom    = hout[na,:] + (eps*sign_out*1.)   # shape (1, M)
    
    message  = (numer/denom) * Rout[na,:]       # shape (D, M)
    
    Rin      = message.sum(axis=1)              # shape (D,)
    
    if debug:
        print("local diff: ", Rout.sum() - Rin.sum())
    # Note: 
    # - local  layer   relevance conservation if bias_factor==1.0 and bias_nb_units==D (i.e. when only one incoming layer)
    # - global network relevance conservation if bias_factor==1.0 and bias_nb_units set accordingly to the total number of lower-layer connections 
    # -> can be used for sanity check
    
    return Rin


**LSTM_bidi.py**

In [ ]:
'''
@author: Leila Arras
@maintainer: Leila Arras
@date: 21.06.2017
@version: 1.0+
@copyright: Copyright (c) 2017, Leila Arras, Gregoire Montavon, Klaus-Robert Mueller, Wojciech Samek
@license: see LICENSE file in repository root
'''

import numpy as np
import pickle
from numpy import newaxis as na
#from code.LSTM.LRP_linear_layer import *


class LSTM_bidi:
    
    def __init__(self, model_path='/content/LRP_for_LSTM/model'):
        """
        Load trained model from file.
        """
        
        """
        # vocabulary
        f_voc     = open('/content/LRP_for_LSTM/model/vocab', 'rb')
        self.voc  = pickle.load(f_voc)
        #print(pickle.load(f_voc))
        print(f_voc)
        f_voc.close()
        """

        # vocabulary
        f_voc     = myvocab
        self.voc  = f_voc
        #print(pickle.load(f_voc))
        print(f_voc)
        #f_voc.close()
        
        # word embeddings
        self.E    = np.load('/content/LRP_for_LSTM/model/embeddings.npy', mmap_mode='r') # shape V*e
        
        # model weights
        f_model   = open('/content/LRP_for_LSTM/model/model', 'rb')
        model     = pickle.load(f_model)
        f_model.close()
        # LSTM left encoder
        self.Wxh_Left  = model["Wxh_Left"]  # shape 4d*e
        self.bxh_Left  = model["bxh_Left"]  # shape 4d 
        self.Whh_Left  = model["Whh_Left"]  # shape 4d*d
        self.bhh_Left  = model["bhh_Left"]  # shape 4d  
        # LSTM right encoder
        self.Wxh_Right = model["Wxh_Right"]
        self.bxh_Right = model["bxh_Right"]
        self.Whh_Right = model["Whh_Right"]
        self.bhh_Right = model["bhh_Right"]   
        # linear output layer
        self.Why_Left  = model["Why_Left"]  # shape C*d
        self.Why_Right = model["Why_Right"] # shape C*d
    

    def set_input(self, w, delete_pos=None):
        """
        Build the numerical input sequence x/x_rev from the word indices w (+ initialize hidden layers h, c).
        Optionally delete words at positions delete_pos.
        """
        T      = len(w)                         # sequence length
        d      = int(self.Wxh_Left.shape[0]/4)  # hidden layer dimension
        e      = self.E.shape[1]                # word embedding dimension
        x      = np.zeros((T, e))
        x[:,:] = self.E[w,:]
        if delete_pos is not None:
            x[delete_pos, :] = np.zeros((len(delete_pos), e))
        
        self.w              = w
        self.x              = x
        self.x_rev          = x[::-1,:].copy()
        
        self.h_Left         = np.zeros((T+1, d))
        self.c_Left         = np.zeros((T+1, d))
        self.h_Right        = np.zeros((T+1, d))
        self.c_Right        = np.zeros((T+1, d))
     
   
    def forward(self):
        """
        Standard forward pass.
        Compute the hidden layer values (assuming input x/x_rev was previously set)
        """
        T      = len(self.w)                         
        d      = int(self.Wxh_Left.shape[0]/4) 
        # gate indices (assuming the gate ordering in the LSTM weights is i,g,f,o):     
        idx    = np.hstack((np.arange(0,d), np.arange(2*d,4*d))).astype(int) # indices of gates i,f,o together
        idx_i, idx_g, idx_f, idx_o = np.arange(0,d), np.arange(d,2*d), np.arange(2*d,3*d), np.arange(3*d,4*d) # indices of gates i,g,f,o separately
          
        # initialize
        self.gates_xh_Left  = np.zeros((T, 4*d))  
        self.gates_hh_Left  = np.zeros((T, 4*d)) 
        self.gates_pre_Left = np.zeros((T, 4*d))  # gates pre-activation
        self.gates_Left     = np.zeros((T, 4*d))  # gates activation
        
        self.gates_xh_Right = np.zeros((T, 4*d))  
        self.gates_hh_Right = np.zeros((T, 4*d)) 
        self.gates_pre_Right= np.zeros((T, 4*d))
        self.gates_Right    = np.zeros((T, 4*d)) 
             
        for t in range(T): 
            self.gates_xh_Left[t]     = np.dot(self.Wxh_Left, self.x[t])        
            self.gates_hh_Left[t]     = np.dot(self.Whh_Left, self.h_Left[t-1]) 
            self.gates_pre_Left[t]    = self.gates_xh_Left[t] + self.gates_hh_Left[t] + self.bxh_Left + self.bhh_Left
            self.gates_Left[t,idx]    = 1.0/(1.0 + np.exp(- self.gates_pre_Left[t,idx]))
            self.gates_Left[t,idx_g]  = np.tanh(self.gates_pre_Left[t,idx_g]) 
            self.c_Left[t]            = self.gates_Left[t,idx_f]*self.c_Left[t-1] + self.gates_Left[t,idx_i]*self.gates_Left[t,idx_g]
            self.h_Left[t]            = self.gates_Left[t,idx_o]*np.tanh(self.c_Left[t])
            
            self.gates_xh_Right[t]    = np.dot(self.Wxh_Right, self.x_rev[t])     
            self.gates_hh_Right[t]    = np.dot(self.Whh_Right, self.h_Right[t-1])
            self.gates_pre_Right[t]   = self.gates_xh_Right[t] + self.gates_hh_Right[t] + self.bxh_Right + self.bhh_Right
            self.gates_Right[t,idx]   = 1.0/(1.0 + np.exp(- self.gates_pre_Right[t,idx]))
            self.gates_Right[t,idx_g] = np.tanh(self.gates_pre_Right[t,idx_g])                 
            self.c_Right[t]           = self.gates_Right[t,idx_f]*self.c_Right[t-1] + self.gates_Right[t,idx_i]*self.gates_Right[t,idx_g]
            self.h_Right[t]           = self.gates_Right[t,idx_o]*np.tanh(self.c_Right[t])
            
        self.y_Left  = np.dot(self.Why_Left,  self.h_Left[T-1])
        self.y_Right = np.dot(self.Why_Right, self.h_Right[T-1])
        self.s       = self.y_Left + self.y_Right
        
        return self.s.copy() # prediction scores
     
              
    def backward(self, w, sensitivity_class):
        """
        Standard gradient backpropagation backward pass.
        Compute the hidden layer gradients by backpropagating a gradient of 1.0 for the class sensitivity_class
        """
        # forward pass
        self.set_input(w)
        self.forward() 
        
        T      = len(self.w)
        d      = int(self.Wxh_Left.shape[0]/4)
        C      = self.Why_Left.shape[0]   # number of classes
        idx    = np.hstack((np.arange(0,d), np.arange(2*d,4*d))).astype(int) # indices of gates i,f,o together
        idx_i, idx_g, idx_f, idx_o = np.arange(0,d), np.arange(d,2*d), np.arange(2*d,3*d), np.arange(3*d,4*d) # indices of gates i,g,f,o separately
        
        # initialize
        self.dx               = np.zeros(self.x.shape)
        self.dx_rev           = np.zeros(self.x.shape)
        
        self.dh_Left          = np.zeros((T+1, d))
        self.dc_Left          = np.zeros((T+1, d))
        self.dgates_pre_Left  = np.zeros((T, 4*d))  # gates pre-activation
        self.dgates_Left      = np.zeros((T, 4*d))  # gates activation
        
        self.dh_Right         = np.zeros((T+1, d))
        self.dc_Right         = np.zeros((T+1, d))
        self.dgates_pre_Right = np.zeros((T, 4*d)) 
        self.dgates_Right     = np.zeros((T, 4*d))  
               
        ds                    = np.zeros((C))
        ds[sensitivity_class] = 1.0
        dy_Left               = ds.copy()
        dy_Right              = ds.copy()
        
        self.dh_Left[T-1]     = np.dot(self.Why_Left.T,  dy_Left)
        self.dh_Right[T-1]    = np.dot(self.Why_Right.T, dy_Right)
        
        for t in reversed(range(T)): 
            self.dgates_Left[t,idx_o]    = self.dh_Left[t] * np.tanh(self.c_Left[t])  # do[t]
            self.dc_Left[t]             += self.dh_Left[t] * self.gates_Left[t,idx_o] * (1.-(np.tanh(self.c_Left[t]))**2) # dc[t]
            self.dgates_Left[t,idx_f]    = self.dc_Left[t] * self.c_Left[t-1]         # df[t]
            self.dc_Left[t-1]            = self.dc_Left[t] * self.gates_Left[t,idx_f] # dc[t-1]
            self.dgates_Left[t,idx_i]    = self.dc_Left[t] * self.gates_Left[t,idx_g] # di[t]
            self.dgates_Left[t,idx_g]    = self.dc_Left[t] * self.gates_Left[t,idx_i] # dg[t]
            self.dgates_pre_Left[t,idx]  = self.dgates_Left[t,idx] * self.gates_Left[t,idx] * (1.0 - self.gates_Left[t,idx]) # d ifo pre[t]
            self.dgates_pre_Left[t,idx_g]= self.dgates_Left[t,idx_g] *  (1.-(self.gates_Left[t,idx_g])**2) # d g pre[t]
            self.dh_Left[t-1]            = np.dot(self.Whh_Left.T, self.dgates_pre_Left[t])
            self.dx[t]                   = np.dot(self.Wxh_Left.T, self.dgates_pre_Left[t])
            
            self.dgates_Right[t,idx_o]    = self.dh_Right[t] * np.tanh(self.c_Right[t])         
            self.dc_Right[t]             += self.dh_Right[t] * self.gates_Right[t,idx_o] * (1.-(np.tanh(self.c_Right[t]))**2) 
            self.dgates_Right[t,idx_f]    = self.dc_Right[t] * self.c_Right[t-1]            
            self.dc_Right[t-1]            = self.dc_Right[t] * self.gates_Right[t,idx_f] 
            self.dgates_Right[t,idx_i]    = self.dc_Right[t] * self.gates_Right[t,idx_g]    
            self.dgates_Right[t,idx_g]    = self.dc_Right[t] * self.gates_Right[t,idx_i]      
            self.dgates_pre_Right[t,idx]  = self.dgates_Right[t,idx] * self.gates_Right[t,idx] * (1.0 - self.gates_Right[t,idx]) 
            self.dgates_pre_Right[t,idx_g]= self.dgates_Right[t,idx_g] *  (1.-(self.gates_Right[t,idx_g])**2) 
            self.dh_Right[t-1]            = np.dot(self.Whh_Right.T, self.dgates_pre_Right[t])
            self.dx_rev[t]                = np.dot(self.Wxh_Right.T, self.dgates_pre_Right[t])
                    
        return self.dx.copy(), self.dx_rev[::-1,:].copy()     
    
                   
    def lrp(self, w, LRP_class, eps=0.001, bias_factor=0.0):
        """
        Layer-wise Relevance Propagation (LRP) backward pass.
        Compute the hidden layer relevances by performing LRP for the target class LRP_class
        (according to the papers:
            - https://doi.org/10.1371/journal.pone.0130140
            - https://doi.org/10.18653/v1/W17-5221 )
        """
        # forward pass
        self.set_input(w)
        self.forward() 
        
        T      = len(self.w)
        d      = int(self.Wxh_Left.shape[0]/4)
        e      = self.E.shape[1] 
        C      = self.Why_Left.shape[0]  # number of classes
        idx    = np.hstack((np.arange(0,d), np.arange(2*d,4*d))).astype(int) # indices of gates i,f,o together
        idx_i, idx_g, idx_f, idx_o = np.arange(0,d), np.arange(d,2*d), np.arange(2*d,3*d), np.arange(3*d,4*d) # indices of gates i,g,f,o separately
        
        # initialize
        Rx       = np.zeros(self.x.shape)
        Rx_rev   = np.zeros(self.x.shape)
        
        Rh_Left  = np.zeros((T+1, d))
        Rc_Left  = np.zeros((T+1, d))
        Rg_Left  = np.zeros((T,   d)) # gate g only
        Rh_Right = np.zeros((T+1, d))
        Rc_Right = np.zeros((T+1, d))
        Rg_Right = np.zeros((T,   d)) # gate g only
        
        Rout_mask            = np.zeros((C))
        Rout_mask[LRP_class] = 1.0  
        
        # format reminder: lrp_linear(hin, w, b, hout, Rout, bias_nb_units, eps, bias_factor)
        Rh_Left[T-1]  = lrp_linear(self.h_Left[T-1],  self.Why_Left.T , np.zeros((C)), self.s, self.s*Rout_mask, 2*d, eps, bias_factor, debug=False)
        Rh_Right[T-1] = lrp_linear(self.h_Right[T-1], self.Why_Right.T, np.zeros((C)), self.s, self.s*Rout_mask, 2*d, eps, bias_factor, debug=False)
        
        for t in reversed(range(T)):
            Rc_Left[t]   += Rh_Left[t]
            Rc_Left[t-1]  = lrp_linear(self.gates_Left[t,idx_f]*self.c_Left[t-1],         np.identity(d), np.zeros((d)), self.c_Left[t], Rc_Left[t], 2*d, eps, bias_factor, debug=False)
            Rg_Left[t]    = lrp_linear(self.gates_Left[t,idx_i]*self.gates_Left[t,idx_g], np.identity(d), np.zeros((d)), self.c_Left[t], Rc_Left[t], 2*d, eps, bias_factor, debug=False)
            Rx[t]         = lrp_linear(self.x[t],        self.Wxh_Left[idx_g].T, self.bxh_Left[idx_g]+self.bhh_Left[idx_g], self.gates_pre_Left[t,idx_g], Rg_Left[t], d+e, eps, bias_factor, debug=False)
            Rh_Left[t-1]  = lrp_linear(self.h_Left[t-1], self.Whh_Left[idx_g].T, self.bxh_Left[idx_g]+self.bhh_Left[idx_g], self.gates_pre_Left[t,idx_g], Rg_Left[t], d+e, eps, bias_factor, debug=False)
            
            Rc_Right[t]  += Rh_Right[t]
            Rc_Right[t-1] = lrp_linear(self.gates_Right[t,idx_f]*self.c_Right[t-1],         np.identity(d), np.zeros((d)), self.c_Right[t], Rc_Right[t], 2*d, eps, bias_factor, debug=False)
            Rg_Right[t]   = lrp_linear(self.gates_Right[t,idx_i]*self.gates_Right[t,idx_g], np.identity(d), np.zeros((d)), self.c_Right[t], Rc_Right[t], 2*d, eps, bias_factor, debug=False)
            Rx_rev[t]     = lrp_linear(self.x_rev[t],     self.Wxh_Right[idx_g].T, self.bxh_Right[idx_g]+self.bhh_Right[idx_g], self.gates_pre_Right[t,idx_g], Rg_Right[t], d+e, eps, bias_factor, debug=False)
            Rh_Right[t-1] = lrp_linear(self.h_Right[t-1], self.Whh_Right[idx_g].T, self.bxh_Right[idx_g]+self.bhh_Right[idx_g], self.gates_pre_Right[t,idx_g], Rg_Right[t], d+e, eps, bias_factor, debug=False)
                   
        return Rx, Rx_rev[::-1,:], Rh_Left[-1].sum()+Rc_Left[-1].sum()+Rh_Right[-1].sum()+Rc_Right[-1].sum()



In [ ]:
a = np.load('/content/LRP_for_LSTM/model/embeddings.npy', mmap_mode='r')
type(a)

numpy.memmap

In [ ]:
a.shape

(19538, 60)

In [ ]:
f_model   = open('/content/LRP_for_LSTM/model/model', 'rb')
model     = pickle.load(f_model)
f_model.close()

In [ ]:
type(model)

dict

In [ ]:
model.shape

{'Whh_Left': array([[-0.0549697 ,  0.10131807,  0.13147646, ..., -0.02586262,
         -0.06841647,  0.09535255],
        [ 0.38883165, -0.12109927, -0.2342312 , ...,  0.19346908,
          0.21419765, -0.38794649],
        [ 0.17802268,  0.08972623,  0.04436522, ...,  0.06123678,
          0.14968273, -0.18579812],
        ...,
        [ 0.18391345, -0.06538825,  0.022021  , ...,  0.09357034,
          0.14026102, -0.06192793],
        [ 0.16286045, -0.01101375, -0.16190973, ...,  0.1560771 ,
         -0.16280361, -0.08899941],
        [-0.07331462, -0.07942951,  0.08378184, ...,  0.03311647,
          0.14240716,  0.09033974]]),
 'Whh_Right': array([[-0.21396473, -0.0030177 ,  0.1533556 , ...,  0.03837524,
          0.32327673, -0.08586927],
        [ 0.37656203, -0.33365756, -0.04590229, ..., -0.00292189,
         -0.10585658,  0.1877445 ],
        [-0.25291473, -0.0333291 ,  0.26158231, ...,  0.20169342,
          0.12286177, -0.19018392],
        ...,
        [-0.07758972,  0.1739

In [ ]:
model.keys()

dict_keys(['Wxh_Left', 'bxh_Left', 'Whh_Left', 'bhh_Left', 'Wxh_Right', 'bxh_Right', 'Whh_Right', 'bhh_Right', 'Why_Left', 'Why_Right'])

In [ ]:
model['Wxh_Left'].shape

(240, 60)

In [ ]:
model['bxh_Left']

array([ 0.45542708,  0.06006528,  0.11785088,  0.0280416 ,  0.0720768 ,
        0.50600201,  0.11075892,  0.13830541,  0.23683275,  0.34989387,
        0.25139314,  0.12887561,  0.14962423,  0.13635223,  0.24726234,
        0.14344472,  0.09248726,  0.36031836,  0.04990448,  0.00526712,
        0.01016608,  0.19329777,  0.16600461,  0.17006224,  0.1740194 ,
        0.03905075,  0.07440132,  0.34844506,  0.04922248, -0.06808653,
        0.06351554,  0.08219348,  0.10147008,  0.52033395,  0.24104194,
        0.09514599,  0.12574963,  0.55344164,  0.12282112,  0.15761299,
        0.37153673,  0.18424208,  0.0877641 ,  0.45423228,  0.13935572,
        0.20485477,  0.15649915,  0.22944976,  0.19999506,  0.17656307,
        0.15394393,  0.00297921,  0.08739223,  0.21466686,  0.07845298,
        0.06799033,  0.1975655 ,  0.04867305,  0.14552975,  0.09707771,
       -0.06831897,  0.12769607,  0.10804035,  0.153965  , -0.13480946,
        0.03848682,  0.17388485,  0.07939359, -0.26035309, -0.01

In [ ]:
model['Whh_Left'].shape

(240, 60)

**heatmap.py**

In [ ]:
'''
@author: Leila Arras
@maintainer: Leila Arras
@date: 21.06.2017
@version: 1.0+
@copyright: Copyright (c) 2017, Leila Arras, Gregoire Montavon, Klaus-Robert Mueller, Wojciech Samek
@license: see LICENSE file in repository root
'''

import matplotlib.pyplot as plt

def rescale_score_by_abs (score, max_score, min_score):
    """
    Normalize the relevance value (=score), accordingly to the extremal relevance values (max_score and min_score), 
    for visualization with a diverging colormap.
    i.e. rescale positive relevance to the range [0.5, 1.0], and negative relevance to the range [0.0, 0.5],
    using the highest absolute relevance for linear interpolation.
    """
    
    # CASE 1: positive AND negative scores occur --------------------
    if max_score>0 and min_score<0:
    
        if max_score >= abs(min_score):   # deepest color is positive
            if score>=0:
                return 0.5 + 0.5*(score/max_score)
            else:
                return 0.5 - 0.5*(abs(score)/max_score)

        else:                             # deepest color is negative
            if score>=0:
                return 0.5 + 0.5*(score/abs(min_score))
            else:
                return 0.5 - 0.5*(score/min_score)   
    
    # CASE 2: ONLY positive scores occur -----------------------------       
    elif max_score>0 and min_score>=0: 
        if max_score == min_score:
            return 1.0
        else:
            return 0.5 + 0.5*(score/max_score)
    
    # CASE 3: ONLY negative scores occur -----------------------------
    elif max_score<=0 and min_score<0: 
        if max_score == min_score:
            return 0.0
        else:
            return 0.5 - 0.5*(score/min_score)    
  
      
def getRGB (c_tuple):
    return "#%02x%02x%02x"%(int(c_tuple[0]*255), int(c_tuple[1]*255), int(c_tuple[2]*255))

     
def span_word (word, score, colormap):
    return "<span style=\"background-color:"+getRGB(colormap(score))+"\">"+word+"</span>"


def html_heatmap (words, scores, cmap_name="bwr"):
    """
    Return word-level heatmap in HTML format,
    with words being the list of words (as string),
    scores the corresponding list of word-level relevance values,
    and cmap_name the name of the matplotlib diverging colormap.
    """
    
    colormap  = plt.get_cmap(cmap_name)
     
    assert len(words)==len(scores)
    max_s     = max(scores)
    min_s     = min(scores)
    
    output_text = ""
    
    for idx, w in enumerate(words):
        score       = rescale_score_by_abs(scores[idx], max_s, min_s)
        output_text = output_text + span_word(w, score, colormap) + " "
    
    return output_text + "\n"


**Define input sequence and relevance target class**

In [ ]:
#from code.LSTM.LSTM_bidi import * 
#from code.util.heatmap import html_heatmap

import codecs
import numpy as np
from IPython.display import display, HTML

In [ ]:
def predict(words):
    """Returns the classifier's predicted class"""
    net                 = LSTM_bidi()                                   # load trained LSTM model
    w_indices           = [net.voc.index(w) for w in words]             # convert input sentence to word IDs
    net.set_input(w_indices)                                            # set LSTM input sequence
    scores              = net.forward()                                 # classification prediction scores
    return np.argmax(scores)            

In [ ]:
#words = df['text'][3]
#words

In [ ]:
words = nltk.word_tokenize(df['text'][3])
words

['indiafightscorona',
 'we',
 'have',
 'covid',
 'testing',
 'laboratories',
 'in',
 'india',
 'and',
 'as',
 'on',
 'th',
 'august',
 'tests',
 'have',
 'been',
 'done',
 'dg',
 'staysafe',
 'indiawillwin']

In [ ]:
#predicted_class = predict(words)                                        # get predicted class

predicted_class = predict(words)                                        # get predicted class
target_class    = predicted_class                                       # define relevance target class 

['the', 'cdc', 'currently', 'reports', 'deaths', 'in', 'general', 'discrepancies', 'death', 'counts', 'between', 'different', 'sources', 'are', 'small', 'and', 'explicable', 'toll', 'stands', 'at', 'roughly', 'people', 'today', 'states', 'reported', 'a', 'rise', 'from', 'last', 'tuesday', 'southern', 'of', 'those', 'politically', 'correct', 'woman', 'almost', 'uses', 'pandemic', 'as', 'excuse', 'not', 'to', 'reuse', 'plastic', 'bag', 'coronavirus', 'nashville', 'indiafightscorona', 'we', 'have', 'covid', 'testing', 'laboratories', 'india', 'on', 'th', 'august', 'tests', 'been', 'done', 'dg', 'staysafe', 'indiawillwin', 'populous', 'can', 'generate', 'large', 'case', 'but', 'if', 'you', 'look', 'new', 'cases', 'per', 'million', 'smaller', 'showing', 'more', 'than', 'california', 'or', 'texas', 'al', 'ar', 'id', 'ks', 'ky', 'la', 'ms', 'nv', 'sc', 'act', 'now', 'found', 'average', 'each', 'person', 'illinois', 'with', 'is', 'infecting', 'other', 'data', 'shows', 'that', 'infection', 'gro

In [ ]:
print(words)
print("\npredicted class:          ",   predicted_class)

['indiafightscorona', 'we', 'have', 'covid', 'testing', 'laboratories', 'in', 'india', 'and', 'as', 'on', 'th', 'august', 'tests', 'have', 'been', 'done', 'dg', 'staysafe', 'indiawillwin']

predicted class:           3


**Compute LRP relevances**

In [ ]:
# LRP hyperparameters:
eps                 = 0.001                                             # small positive number
bias_factor         = 0.0                                               # recommended value
 
net                 = LSTM_bidi()                                       # load trained LSTM model

w_indices           = [net.voc.index(w) for w in words]                 # convert input sentence to word IDs
Rx, Rx_rev, R_rest  = net.lrp(w_indices, target_class, eps, bias_factor)# perform LRP
R_words             = np.sum(Rx + Rx_rev, axis=1)                       # compute word-level LRP relevances

scores              = net.s.copy()                                      # classification prediction scores

['the', 'cdc', 'currently', 'reports', 'deaths', 'in', 'general', 'discrepancies', 'death', 'counts', 'between', 'different', 'sources', 'are', 'small', 'and', 'explicable', 'toll', 'stands', 'at', 'roughly', 'people', 'today', 'states', 'reported', 'a', 'rise', 'from', 'last', 'tuesday', 'southern', 'of', 'those', 'politically', 'correct', 'woman', 'almost', 'uses', 'pandemic', 'as', 'excuse', 'not', 'to', 'reuse', 'plastic', 'bag', 'coronavirus', 'nashville', 'indiafightscorona', 'we', 'have', 'covid', 'testing', 'laboratories', 'india', 'on', 'th', 'august', 'tests', 'been', 'done', 'dg', 'staysafe', 'indiawillwin', 'populous', 'can', 'generate', 'large', 'case', 'but', 'if', 'you', 'look', 'new', 'cases', 'per', 'million', 'smaller', 'showing', 'more', 'than', 'california', 'or', 'texas', 'al', 'ar', 'id', 'ks', 'ky', 'la', 'ms', 'nv', 'sc', 'act', 'now', 'found', 'average', 'each', 'person', 'illinois', 'with', 'is', 'infecting', 'other', 'data', 'shows', 'that', 'infection', 'gro

In [ ]:
print ("prediction scores:        ",   scores)
print ("\nLRP target class:         ", target_class)
print ("\nLRP relevances:")
for idx, w in enumerate(words):
    print ("\t\t\t" + "{:8.2f}".format(R_words[idx]) + "\t" + w)
print ("\nLRP heatmap:")    
display(HTML(html_heatmap(words, R_words)))

prediction scores:         [-4.43595332 -1.39615754  1.29417753  1.7439628  -0.06750484]

LRP target class:          3

LRP relevances:
			    0.24	indiafightscorona
			   -0.22	we
			   -0.12	have
			    1.12	covid
			    0.55	testing
			    0.22	laboratories
			   -0.15	in
			   -0.06	india
			    0.01	and
			   -0.03	as
			    0.33	on
			    0.22	th
			    0.25	august
			   -0.03	tests
			   -0.01	have
			    0.07	been
			   -0.10	done
			    0.57	dg
			    0.16	staysafe
			   -0.16	indiawillwin

LRP heatmap:


In [ ]:
# How to sanity check global relevance conservation:
bias_factor        = 1.0                                             # value to use for sanity check
Rx, Rx_rev, R_rest = net.lrp(w_indices, target_class, eps, bias_factor)
R_tot              = Rx.sum() + Rx_rev.sum() + R_rest.sum()          # sum of all "input" relevances

print(R_tot)       ;    print("Sanity check passed? ", np.allclose(R_tot, net.s[target_class]))

1.7439627997789005
Sanity check passed?  True


**Compute SA/GI relevances**

In [ ]:
net              = LSTM_bidi()                                       # load trained LSTM model

w_indices        = [net.voc.index(w) for w in words]                 # convert input sentence to word IDs
Gx, Gx_rev       = net.backward(w_indices, target_class)             # perform gradient backpropagation
R_words_SA       = (np.linalg.norm(Gx + Gx_rev, ord=2, axis=1))**2   # compute word-level Sensitivity Analysis relevances
R_words_GI       = ((Gx + Gx_rev)*net.x).sum(axis=1)                 # compute word-level GradientxInput relevances

scores           = net.s.copy()                                      # classification prediction scores 

['the', 'cdc', 'currently', 'reports', 'deaths', 'in', 'general', 'discrepancies', 'death', 'counts', 'between', 'different', 'sources', 'are', 'small', 'and', 'explicable', 'toll', 'stands', 'at', 'roughly', 'people', 'today', 'states', 'reported', 'a', 'rise', 'from', 'last', 'tuesday', 'southern', 'of', 'those', 'politically', 'correct', 'woman', 'almost', 'uses', 'pandemic', 'as', 'excuse', 'not', 'to', 'reuse', 'plastic', 'bag', 'coronavirus', 'nashville', 'indiafightscorona', 'we', 'have', 'covid', 'testing', 'laboratories', 'india', 'on', 'th', 'august', 'tests', 'been', 'done', 'dg', 'staysafe', 'indiawillwin', 'populous', 'can', 'generate', 'large', 'case', 'but', 'if', 'you', 'look', 'new', 'cases', 'per', 'million', 'smaller', 'showing', 'more', 'than', 'california', 'or', 'texas', 'al', 'ar', 'id', 'ks', 'ky', 'la', 'ms', 'nv', 'sc', 'act', 'now', 'found', 'average', 'each', 'person', 'illinois', 'with', 'is', 'infecting', 'other', 'data', 'shows', 'that', 'infection', 'gro

In [ ]:
print ("prediction scores:       ",   scores)
print ("\nSA/GI target class:      ", target_class)
print ("\nSA relevances:")
for idx, w in enumerate(words):
    print ("\t\t\t" + "{:8.2f}".format(R_words_SA[idx]) + "\t" + w)
print ("\nSA heatmap:")    
display(HTML(html_heatmap(words, R_words_SA)))
print ("\nGI relevances:")
for idx, w in enumerate(words):
    print ("\t\t\t" + "{:8.2f}".format(R_words_GI[idx]) + "\t" + w)
print ("\nGI heatmap:")    
display(HTML(html_heatmap(words, R_words_GI)))

prediction scores:        [-4.43595332 -1.39615754  1.29417753  1.7439628  -0.06750484]

SA/GI target class:       3

SA relevances:
			    2.10	indiafightscorona
			    0.89	we
			    1.44	have
			    0.85	covid
			    0.28	testing
			    1.11	laboratories
			    0.23	in
			    0.02	india
			    0.01	and
			    0.05	as
			    0.19	on
			    0.17	th
			    0.08	august
			    0.09	tests
			    0.26	have
			    0.51	been
			    0.90	done
			    0.07	dg
			    0.03	staysafe
			    0.17	indiawillwin

SA heatmap:



GI relevances:
			    0.12	indiafightscorona
			   -0.01	we
			   -0.03	have
			    0.17	covid
			   -0.03	testing
			    0.15	laboratories
			    0.18	in
			    0.09	india
			    0.02	and
			    0.08	as
			    0.06	on
			    0.06	th
			    0.09	august
			    0.09	tests
			    0.10	have
			    0.12	been
			    0.07	done
			    0.03	dg
			   -0.01	staysafe
			    0.10	indiawillwin

GI heatmap:
